### Imports

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from google.cloud import storage
from io import BytesIO

#### Variables

In [3]:
classes = 'Boat'

Download darknet and create darknet gpu

In [ ]:
#Those commands are # as downloads 250MB files and only need to be run the first time

#Clone darknet
#!git clone https://github.com/pjreddie/darknet.git

# Build gpu version darknet
#!cd darknet && sed '1 s/^.*$/GPU=1/; 2 s/^.*$/CUDNN=1/' -i Makefile

# -j <The # of cpu cores to use>. Change 4 to fit your environment.
#!cd darknet && make -j 4 -s
#!cp darknet/darknet darknet_gpu


In [ ]:
#Donwloads pre train weights around 200MB
#!wget -q https://pjreddie.com/media/files/darknet53.conv.74

#### Paths

In [4]:
RAW_DATA_PATH = '/home/jupyter/raw_data/'
DATA_PATH = '/home/jupyter/data/'

FRAME_PATH = os.path.join(os.getcwd(), 'images')  # .jpg
MASK_PATH = os.path.join(os.getcwd(), 'labels')  # .txt
METADATA_PATH = os.path.join(os.getcwd(), 'metadata') # .txt

# YOLOv3 config file directory
CONFIG_PATH = os.path.join(os.getcwd(), 'cfg')

# YOLOv3 training weights will be saved here
BACKUP_PATH = os.path.join(os.getcwd(), 'backup')

DATA_EXTENSION_FILE_PATH = os.path.join(CONFIG_PATH, 'ih_fp.data')


To be able to train the model we need to indicate the paths of data needed

In [5]:
with open(DATA_EXTENSION_FILE_PATH, 'w') as f:
    contents = """classes= 1
train  = {}
valid  = {}
names  = {}
backup = {}
    """.format(os.path.join(METADATA_PATH, "tr_list.txt"),
               os.path.join(METADATA_PATH, "val_list.txt"),
               os.path.join(CONFIG_PATH, 'ih_fp.names'),
               BACKUP_PATH)
    f.write(contents)

In [6]:
!cat cfg/ih_fp.data

classes= 1
train  = /home/jupyter/metadata/tr_list.txt
valid  = /home/jupyter/metadata/val_list.txt
names  = /home/jupyter/cfg/ih_fp.names
backup = /home/jupyter/backup
    

We add classes to configuration names file

In [7]:
!echo classes > cfg/ih_fp.names

We are ready to train the model. While training all logs are saved in train_log.txt file. This allow us to while training check how is doing. (See notebook 4)

In [13]:
!cd darknet

In [ ]:
#Train
!./darknet_gpu detector train cfg/ih_fp.data cfg/yolov3_ih_fp.cfg darknet53.conv.74 -i 0 | tee train_log.txt

layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   416 x 416 x   3   ->   416 x 416 x  32  0.299 BFLOPs
    1 conv     64  3 x 3 / 2   416 x 416 x  32   ->   208 x 208 x  64  1.595 BFLOPs
    2 conv     32  1 x 1 / 1   208 x 208 x  64   ->   208 x 208 x  32  0.177 BFLOPs
    3 conv     64  3 x 3 / 1   208 x 208 x  32   ->   208 x 208 x  64  1.595 BFLOPs
    4 res    1                 208 x 208 x  64   ->   208 x 208 x  64
    5 conv    128  3 x 3 / 2   208 x 208 x  64   ->   104 x 104 x 128  1.595 BFLOPs
    6 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64  0.177 BFLOPs
    7 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128  1.595 BFLOPs
    8 res    5                 104 x 104 x 128   ->   104 x 104 x 128
    9 conv     64  1 x 1 / 1   104 x 104 x 128   ->   104 x 104 x  64  0.177 BFLOPs
   10 conv    128  3 x 3 / 1   104 x 104 x  64   ->   104 x 104 x 128  1.595 BFLOPs
   11 res    8                 104 x 